In [1]:
from netCDF4 import Dataset
import matplotlib as mpl
import matplotlib.pyplot as plt
import cartopy.crs as crs
import cartopy.feature as cf
import numpy as np
import xarray as xr
import math
import geopandas as gpd
from wrf import (to_np, getvar, get_cartopy, cartopy_xlim, cartopy_ylim, latlon_coords, ALL_TIMES)


In [2]:
# Open the NetCDF file
ncfile = Dataset("../Temp.2023-09-25.nc")
temp = getvar(ncfile, "T2", timeidx = ALL_TIMES)

# Get the latitude and longitude points
lats, lons = latlon_coords(temp)

# Get the cartopy mapping object
cart_proj = get_cartopy(temp)
xlim = cartopy_xlim(temp)
ylim = cartopy_ylim(temp)

In [4]:
#ncfile = Dataset("../LANDMASK.nc")
#mask = getvar(ncfile, "LANDMASK")
dataset = xr.open_dataset("../2023-10-05/Temp.2023-10-05.nc")
t = dataset['T2']
dataset = xr.open_dataset('masking/mask.nc')
mask = dataset['__xarray_dataarray_variable__']
m = to_np(mask).astype('float')
m[m==0] = np.nan
tc = (t-273.15)*m


In [5]:
# Create temp color map
nws_precip_colors = [
    "#fdfdfd",  # 10.00+
    "#04e9e7",  # 0.01 - 0.10 inches
    "#019ff4",  # 0.10 - 0.25 inches
    "#0300f4",  # 0.25 - 0.50 inches
    "#02fd02",  # 0.50 - 0.75 inches
    "#01c501",  # 0.75 - 1.00 inches
    "#008e00",  # 1.00 - 1.50 inches
    "#fdf802",  # 1.50 - 2.00 inches
    "#e5bc00",  # 2.00 - 2.50 inches
    "#fd9500",  # 2.50 - 3.00 inches
    "#fd0000",  # 3.00 - 4.00 inches
    "#d40000",  # 4.00 - 5.00 inches
    "#bc0000",  # 5.00 - 6.00 inches
    "#f800fd",  # 6.00 - 8.00 inches
    "#9854c6"   # 8.00 - 10.00 inches
]
precip_colormap = mpl.colors.ListedColormap(nws_precip_colors) #mpl.colormaps["jet"]

In [52]:
# Create a figure    
def plot(t2,r,t):
    fig = plt.figure(figsize=(8,6))
    # Set the GeoAxes to the projection used by WRF
    ax = plt.axes(projection=cart_proj)

    levels = [10, 12, 14, 16, 18, 20, 22, 24, 26, 28, 30, 32, 34, 36, 38, 40]
    norm = mpl.colors.BoundaryNorm(levels, 15)

    # Download and add the states and coastlines
    ax.add_feature(cf.BORDERS)
    ax.coastlines('50m', linewidth=0.8)

#    plt.contourf(to_np(lons), to_np(lats), to_np(t2), levels,
    plt.pcolormesh(lons, lats, t2, vmin = 8, vmax = 34,
             transform=crs.PlateCarree(),
             cmap=mpl.colormaps["gist_ncar"])#, norm=norm)

    # Add a color bar
    plt.colorbar(ax=ax, shrink=.9, ticks = levels)

    # Set the map bounds
    ax.set_xlim(xlim)
    ax.set_ylim(ylim)

    # Add the gridlines
    ax.gridlines(color='black', linestyle='dotted')

    plt.title('Temperature of ensemble '+str(r)+' at week '+str(t+1)+' (C)')

    #plt.show()
    plt.savefig('T_t'+str(t+1)+'_e'+str(r)+'.png', dpi=200)
    plt.close()

def plot_trend(t2,r):
    fig = plt.figure(figsize=(8,6))
    # Set the GeoAxes to the projection used by WRF
    ax = plt.axes(projection=cart_proj)

    #levels = [-6.5 -6 -5.5 -5 -4.5 -4 -3.5 -3 -2.5 -2 -1.5 -1, -0.5, 0, 0.5, 1]
    levels = np.arange(-14, 15, 1).tolist()
    norm = mpl.colors.BoundaryNorm(levels, 28)

    # Download and add the states and coastlines
    ax.add_feature(cf.BORDERS)
    ax.coastlines('50m', linewidth=0.8)

#    plt.contourf(to_np(lons), to_np(lats), to_np(t2), levels,
#             transform=crs.PlateCarree(),
#             cmap=mpl.colormaps["coolwarm"], norm=norm)
#    plt.contourf(to_np(lons), to_np(lats), to_np(t2),
    plt.pcolormesh(lons, lats, t2,
             transform=crs.PlateCarree(),
             cmap=mpl.colormaps["Spectral_r"])
#             cmap=mpl.colormaps["YlGnBu_r"])

    # Add a color bar
    plt.colorbar(ax=ax, shrink=.9, ticks = [-2, -1.5, -1, -0.5, 0])

    # Set the map bounds
    ax.set_xlim(xlim)
    ax.set_ylim(ylim)

    # Add the gridlines
    ax.gridlines(color='black', linestyle='dotted')

    plt.title('Temperature trend of ensemble '+str(r)+' (C)')

    #plt.show()
    plt.savefig('DT_e'+str(r)+'.png', dpi=200)
    plt.close()

In [53]:
r = 'median'
for t in range(0,6):
    print(t)
    t2 = tc.isel(Time=t)
#    plot(t2.median(dim='record'),r,t)
t5 = tc.isel(Time=5)
t0 = tc.isel(Time=0)
dt = t5.median(dim='record')-t0.median(dim='record')
plot_trend(dt/5,r)


0
1
2
3
4
5


/Users/luongtm/miniforge3/lib/python3.10/site-packages/cartopy/mpl/geoaxes.py:1781: UserWarning: The input coordinates to pcolormesh are interpreted as cell centers, but are not monotonically increasing or decreasing. This may lead to incorrectly calculated cell edges, in which case, please supply explicit cell edges to pcolormesh.
  result = super().pcolormesh(*args, **kwargs)


In [18]:
print(xlim)

[-1532000.00097559, 1531998.765921907]
